#  Reconstructing the path
In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample_small.txt

Format: user_id \t follower_id

You can start with the code described in "Starter..." (see the next self-reading).

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path “12 -> 42 -> 34” should be printed as:

    12,42,34

Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time.

The result on the sample dataset:

    12,422,53,52,107,20,23,274,34

If you want to deploy the environment on your own machine, please use bigdatateam/spark-course1 Docker container.

In [1]:
BEGIN = 12
END = 34
N = 400  # number of partitions

In [3]:
# Python2 on grader, Python3 in docker
# import sys
# if sys.version_info.major == 3:
#     import os
#     os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python'
#     os.environ['PYTHONHASHSEED'] = '42'
#    unicode = str

In [4]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [5]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

reverse_edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
edges = reverse_edges.map(lambda e: (e[1], e[0])).partitionBy(N).cache()

In [7]:
# edges.take(5)

In [ ]:
# I stored the full path in items, but the grader said it takes
# too much memory. Now storing only the previous node.

In [61]:
def step(item):
    (prev_v, ((prev_d, _), next_v)) = item
    return (next_v, (prev_d + 1, prev_v))

In [62]:
def complete(item):
    (v, (old_item, new_item)) = item
    return (v, old_item if old_item is not None else new_item)

In [63]:
seed_distances = sc.parallelize([(BEGIN, (0, None))]).partitionBy(N)

In [64]:
# candidates = distances.join(edges, N).map(step)
# candidates.cache()
# candidates.take(5)

[(126, (1, 12)), (380, (1, 12)), (422, (1, 12)), (648, (1, 12)), (23, (6, 20))]

In [65]:
# new_distances = distances.fullOuterJoin(candidates, N).map(complete, True).cache()
# new_distances.take(5)

[(12, (0, [12])),
 (13, (4, [12, 422, 53, 31, 13])),
 (15, (5, [12, 422, 53, 52, 107, 15])),
 (15, (5, [12, 422, 53, 52, 107, 15])),
 (15, (5, [12, 422, 53, 52, 107, 15]))]

In [79]:
end_found = []

d = 0
distances = seed_distances

while True:
    # print("d =", d, ", some distances are:", distances.take(5))    print("d =", d, ", count(distances) = ", distances.count())
    print("d =", d, ", count(distances) = ", distances.count())
    candidates = distances.join(edges, N).map(step)
    new_distances = distances.fullOuterJoin(candidates, N).map(complete, True).persist()
    count = new_distances.filter(lambda i: i[1][0] == d + 1).count()
    if count > 0:
        d += 1
        distances = new_distances
        end_found = new_distances.lookup(END)
        if len(end_found):
            break
    else:
        break

d = 0 , count(distances) =  1
d = 1 , count(distances) =  5
d = 2 , count(distances) =  6
d = 3 , count(distances) =  13
d = 4 , count(distances) =  15
d = 5 , count(distances) =  18
d = 6 , count(distances) =  23
d = 7 , count(distances) =  34


In [77]:
path = []

cur_v = END
while cur_v:
    path.append(cur_v)
    back = distances.lookup(cur_v)
    if not len(back):
        break
    cur_v = back[0][1]


path.reverse()
strpath = [str(v) for v in path]
','.join(strpath)


'12,422,53,52,107,20,23,274,34'

In [78]:
# distances.count()

73